In [1]:
from src.utils import get_sedona
import geopandas as gpd
from keplergl import KeplerGl
import pandas as pd

In [2]:
config_path = 'src/config.json'
sedona = get_sedona(config_path)

25/03/16 23:19:29 WARN Utils: Your hostname, cirowhois resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
25/03/16 23:19:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1


:: loading settings :: url = jar:file:/home/cirowhois/CODE/etl-criminalidade-sp/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cirowhois/.ivy2/cache
The jars for the packages stored in: /home/cirowhois/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-23d8889e-f1f9-4502-a4b6-ea7b788774ec;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.0 in central
	found org.apache.sedona#sedona-common;1.7.0 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.g

# MUNICÍPIOS

In [4]:
data_path = 'src/data/gold/tb_name=gold_crimes_mun/YEAR_INFO=2024'
data = sedona.read.parquet(data_path)
gdf = data.toPandas()
gdf = gpd.GeoDataFrame(gdf,geometry='geom', crs=4326)

25/03/16 21:42:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
gdf_2024_mun = gdf[['ID_MUN','MUN','STATE','TOTAL_CRIMES','FURTO_ROUBO_DE_CELULAR','FURTO_ROUBO_DE_VEICULO','POP_TOTAL','TOTAL_CRIME_INDEX','CELLPHONE_CRIME_INDEX','VEHICLE_CRIME_INDEX','geom']]



mapa_mun = KeplerGl(height=600)
mapa_mun.add_data(data=gdf_2024_mun.to_json(), name="São Paulo - Crimes")


mapa_mun

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'São Paulo - Crimes': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature"…

In [ ]:
mapa_mun.save_to_html(file_name='html/kepler/mapa_mun_2024.html')

Map saved to html/kepler/mapa_mun.html!


# SCS

In [10]:
data_path = 'src/data/gold/tb_name=gold_crimes_scs/YEAR_INFO=2024'
data = sedona.read.parquet(data_path).filter('ID_MUN in (3509502,3520509,3556701,3556206,3523404,3532009,3537107,3524709,3548005,3519055,3515152,3503802,3512803,3536505,3501608,3533403,3545803,3552403,3519071,3531803,3536505)')
gdf = data.toPandas()
gdf = gpd.GeoDataFrame(gdf,geometry='geom', crs=4326)

In [11]:
gdf_2024_scs = gdf[['ID_SC','MUN','STATE','TOTAL_CRIMES','FURTO_ROUBO_DE_CELULAR','FURTO_ROUBO_DE_VEICULO','POP_TOTAL','TOTAL_CRIME_INDEX','CELLPHONE_CRIME_INDEX','VEHICLE_CRIME_INDEX','geom']]

mapa_scs = KeplerGl(height=600)
mapa_scs.add_data(data=gdf_2024_scs.to_json(), name="RMC - Crimes")

mapa_scs

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'RMC - Crimes': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "pro…

In [ ]:
mapa_scs.save_to_html(file_name='html/kepler/mapa_scs_rmc_2024.html')

Map saved to html/kepler/mapa_scs.html!


# OCORRÊNCIAS

In [15]:
data_path = 'src/data/gold/tb_name=gold_crimes/'
data = sedona.read.parquet(data_path) \
    .filter('ID_MUN in (3509502) AND YEAR_INFO > 2021') \
    .drop('DATE_PROCESSED')

gdf = data.toPandas()
gdf = gpd.GeoDataFrame(gdf,geometry='geom', crs=4326)
gdf['DATE_CRIME'] = gdf['DAY_INFO'].astype(str) + '/' + gdf['MONTH_INFO'].astype(str) + '/' + gdf['YEAR_INFO'].astype(str)
gdf['DATE_CRIME'] = pd.to_datetime(gdf['DATE_CRIME'], format='%d/%m/%Y', errors='coerce')
gdf['DATE_CRIME'] = gdf['DATE_CRIME'].dt.strftime('%Y-%m-%d %H:%M:%S')
gdf['lat'] = gdf.geom.y
gdf['long'] = gdf.geom.x

data_border = sedona.read.parquet('src/data/silver/tb_name=silver_mun').select('ID_MUN','MUN','STATE','geom').filter('ID_MUN in (3509502)')
gdf_border = data_border.toPandas()
gdf_border = gpd.GeoDataFrame(gdf_border,geometry='geom',crs=4326)

In [16]:
mapa_geral = KeplerGl(height=800)
mapa_geral.add_data(data=gdf.to_json(), name="Crimes - Campinas/SP")
mapa_geral.add_data(data=gdf_border.to_json(), name="Limite - Campinas/SP")

mapa_geral

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Crimes - Campinas/SP': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Featur…

In [ ]:
mapa_geral.save_to_html(file_name='html/kepler/mapa_geral_campinas.html')

Map saved to html/kepler/mapa_geral.html!
